![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use AutoAI RAG and Chroma to create a pattern and get information from `ibm-watsonx-ai` SDK documentation

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate the usage of IBM AutoAI RAG. The AutoAI RAG experiment conducted in this notebook uses data scraped from the `ibm-watsonx-ai` SDK documentation.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The learning goals of this notebook are:

- Create an AutoAI RAG job that will find the best RAG pattern based on provided data


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [RAG Optimizer definition](#definition)
- [RAG Experiment run](#run)
- [RAG Patterns comparison and testing](#comparison)
- [Historical runs](#runs)
- [Clean up](#cleanup)
- [Summary and next steps](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the required modules and dependencies

In [ ]:
!pip install -U 'ibm-watsonx-ai[rag]>=1.1.11' | tail -n 1
!pip install -U "langchain_community>=0.3,<0.4" | tail -n 1

### Defining the watsonx.ai credentials
This cell defines the credentials required to work with the watsonx.ai Runtime service.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your watsonx.ai api key (hit enter): "),
)

### Defining the project id
The foundation model requires a project id that provides the context for the call. We will try to obtain the id directly from the project in which this notebook runs. If this fails, you'll have to provide the project id.

In [2]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

Create an instance of APIClient with authentication details.

In [2]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials=credentials, project_id=project_id)

<a id="definition"></a>

## RAG Optimizer definition

### Defining a connection to training data

Upload training data to a COS bucket and then define a connection to this file. This example uses the `ModelInference` description from the [`ibm_watsonx_ai`](https://ibm.github.io/watsonx-ai-python-sdk/fm_model_inference.html) documentation.

In [4]:
from langchain_community.document_loaders import WebBaseLoader

url = "https://ibm.github.io/watsonx-ai-python-sdk/fm_model_inference.html"

docs = WebBaseLoader(url).load()
model_inference_content = docs[0].page_content

The code in the next cell uploads training data to the bucket.

In [5]:
import os

document_filename = "ModelInference.txt"

if not os.path.isfile(document_filename):
    with open(document_filename, "w") as file:
        file.write(model_inference_content)

document_asset_details = client.data_assets.create(name=document_filename, file_path=document_filename)

document_asset_id = client.data_assets.get_id(document_asset_details)
document_asset_id

Creating data asset...
SUCCESS


'dc17d103-f772-4ea9-965a-b4d8a7093bf0'

Define a connection to training data.

In [6]:
from ibm_watsonx_ai.helpers import DataConnection

input_data_references = [DataConnection(data_asset_id=document_asset_id)]

### Defining a connection to test data

Upload a `json` file that will be used for benchmarking to COS and then define a connection to this file. This example uses content from the [`ibm_watsonx_ai`](https://ibm.github.io/watsonx-ai-python-sdk/index.html) SDK documentation.

In [7]:
benchmarking_data_IBM_page_content = [
    {
        "question": "What is path to ModelInference class?",
        "correct_answer": "ibm_watsonx_ai.foundation_models.inference.ModelInference",
        "correct_answer_document_ids": [
            "ModelInference.txt"
        ]
    },
    {
        "question": "What is method for get model inferance details?",
        "correct_answer": "get_details()",
        "correct_answer_document_ids": [
            "ModelInference.txt"
        ]
    }
]

The code in the next cell uploads testing data to the bucket as a `json` file.

In [8]:
import json

test_filename = "benchmarking_data_ModelInference.json"

if not os.path.isfile(test_filename):
    with open(test_filename, "w") as json_file:
        json.dump(benchmarking_data_IBM_page_content, json_file, indent=4)

test_asset_details = client.data_assets.create(name=test_filename, file_path=test_filename)

test_asset_id = client.data_assets.get_id(test_asset_details)
test_asset_id

Creating data asset...
SUCCESS


'887c3f64-eff2-47eb-a279-03621405880a'

Define connection information to testing data.

In [9]:
test_data_references = [DataConnection(data_asset_id=test_asset_id)]

### RAG Optimizer configuration

Provide the input information for AutoAI RAG optimizer:
- `name` - experiment name
- `description` - experiment description
- `max_number_of_rag_patterns` - maximum number of RAG patterns to create
- `optimization_metrics` - target optimization metrics

In [10]:
from ibm_watsonx_ai.experiment import AutoAI

experiment = AutoAI(credentials, project_id=project_id)

rag_optimizer = experiment.rag_optimizer(
    name='AutoAI RAG run - ModelInference documentation',
    description="AutoAI RAG Optimizer on ibm_watsonx_ai ModelInference documentation",
    max_number_of_rag_patterns=4,
    optimization_metrics=[AutoAI.RAGMetrics.ANSWER_CORRECTNESS]
)

Configuration parameters can be retrieved via `get_params()`.

In [11]:
rag_optimizer.get_params()

{'name': 'AutoAI RAG run - ModelInference documentation',
 'description': 'AutoAI RAG Optimizer on ibm_watsonx_ai ModelInference documentation',
 'max_number_of_rag_patterns': 8,
 'optimization_metrics': ['answer_correctness']}

<a id="run"></a>
## RAG Experiment run

Call the `run()` method to trigger the AutoAI RAG experiment. You can either use interactive mode (synchronous job) or background mode (asynchronous job) by specifying `background_mode=True`.

In [12]:
run_details = rag_optimizer.run(
    input_data_references=input_data_references,
    test_data_references=test_data_references,
    background_mode=False
)



##############################################

Running 'b6315fa9-b65a-4fcb-a4e1-bf717d32c359'

##############################################


pending...
running........................
completed
Training of 'b6315fa9-b65a-4fcb-a4e1-bf717d32c359' finished successfully.


You can use the `get_run_status()` method to monitor AutoAI RAG jobs in background mode.

In [13]:
rag_optimizer.get_run_status()

'completed'

<a id="comparison"></a>
## Comparison and testing of RAG Patterns

You can list the trained patterns and information on evaluation metrics in the form of a Pandas DataFrame by calling the `summary()` method. You can use the DataFrame to compare all discovered patterns and select the one you like for further testing.

In [14]:
summary = rag_optimizer.summary()
summary

,mean_answer_correctness,mean_faithfulness,mean_context_correctness,chunking.chunk_size,embeddings.model_id,vector_store.distance_metric,retrieval.method,retrieval.number_of_chunks,generation.model_id
Pattern_Name,,,,,,,,,
Pattern1,1.0,0.65940,1.0,512,ibm/slate-125m-english-rtrvr,euclidean,window,5,meta-llama/llama-3-70b-instruct
Pattern4,1.0,0.85365,1.0,1024,intfloat/multilingual-e5-large,cosine,window,5,meta-llama/llama-3-70b-instruct
Pattern6,1.0,0.73705,1.0,1024,intfloat/multilingual-e5-large,euclidean,window,3,meta-llama/llama-3-70b-instruct
Pattern8,1.0,0.87500,1.0,1024,ibm/slate-125m-english-rtrvr,euclidean,window,3,meta-llama/llama-3-70b-instruct
Pattern2,0.5,0.29950,1.0,1024,intfloat/multilingual-e5-large,euclidean,window,5,ibm/granite-13b-chat-v2
Pattern5,0.5,0.61920,1.0,1024,intfloat/multilingual-e5-large,cosine,window,3,mistralai/mixtral-8x7b-instruct-v01
Pattern7,0.5,0.29030,1.0,1024,ibm/slate-125m-english-rtrvr,cosine,window,5,ibm/granite-13b-chat-v2
Pattern3,0.0,0.71430,1.0,1024,intfloat/multilingual-e5-large,euclidean,simple,5,ibm/granite-13b-chat-v2


Additionally, you can pass the `scoring` parameter to the summary method, to filter RAG patterns starting with the best.

```python
summary = rag_optimizer.summary(scoring="faithfulness")
```

### Get selected pattern

Get the RAGPattern object from the RAG Optimizer experiment. By default, the RAGPattern of the best pattern is returned.

In [15]:
best_pattern_name = summary.index.values[0]
print('Best pattern is:', best_pattern_name)

best_pattern = rag_optimizer.get_pattern(pattern_name="Pattern1")
best_pattern

Best pattern is: Pattern1


The pattern details can be retrieved by calling the `get_pattern_details` method:

```python
rag_optimizer.get_pattern_details(pattern_name='Pattern2')
```

### Create the index/collection

Build solution on the best pattern, with additional document indexing.

You can check which `index_name` you are working on:

```python
best_pattern.vector_store._index_name
```

In [16]:
urls = [
    "https://ibm.github.io/watsonx-ai-python-sdk/fm_embeddings.html",
    "https://ibm.github.io/watsonx-ai-python-sdk/fm_custom_models.html",
    "https://ibm.github.io/watsonx-ai-python-sdk/fm_text_extraction.html"
]
docs_list = WebBaseLoader(urls).load()
doc_splits = best_pattern.chunker.split_documents(docs_list)

In [17]:
best_pattern.indexing_function(doc_splits)

['1d66cc8bdf47bf8a6feb581bad2da1b3a84efe6e315b704e257c4694503262fa',
 '41d202e18173f3de8e6fe2487f94be3ba343a0a1ec0ace31a08d15d43df2cdda',
 '2601c867f6ec009dba86bb56b0736e5fcfc0dc6b4dbfac76ca3658d7adb78841',
 'aa0c2db7aae540ae0d65378060f96cd903a6f1dff07cbc4b69a1c19e5e022eaa',
 '345c1341dbf0413714aba8a5cfc24df26fc927fdd9221aa24db9331172b3730c',
 'c8ee6e3a9c29e5b8095b9934634b74d7850c385fba543008de11ceff499205bc',
 '8f2d0cd3e48b38ffca8c8781fd701de6f4d15ef3733f73dfda432a9855976a1f',
 '21a8c1f5d6fe14ad136ff85923b8c128c1c21a8eaf8bebefb113f4b2b7df831f',
 '05178f9c7c9f54e1632df71ef376b09dad7ec74c24f9e820e5a81eb3a9324a55',
 '38cf82fa385db9287d2caa7e64e69935d7cd391fba1f642fc3aba96d8909cd57',
 '8f6c8405f9cf229ae0210881e31c53f88942d836a78209f856c12db676cff120',
 '3e4381579adc876a54287f2f175090558a15c1394820851341bc4803f53dc7bf',
 'c5ba02114835949db97af1e54a1617178a02f9407126c199f3f1abe86ca13e02',
 '666a2c2447484692e4efcdf184ce58668e468b72e1008f46390c5cee39c92843',
 'a831b5576c6019cf5268426fe970e71a

Query the RAGPattern locally, to test it.

In [18]:
questions = ["How to add Task Credentials?"]

payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [
        {
            "values": questions,
            "access_token": client.service_instance._get_token()
        }
    ]
}

best_pattern.inference_function()(payload)

{'predictions': [{'fields': ['answer', 'reference_documents'],
   'values': [["\n\nTo add Task Credentials, you can use the `store` method of the `task_credentials` object. First, initialize the `APIClient` object with your credentials, then set the default project or space ID. Next, use the `list` method to check if there are any existing task credentials. If the list is empty, you can create new task credentials by calling the `store` method. You can also use the `get_details` method to get the status of available task credentials.\n\nHere's an example:\n```\nfrom ibm_watsonx_ai import APIClient\n\nclient = APIClient(credentials)\nclient.set.default_project(project_id=project_id)\n\n# Check if there are existing task credentials\nclient.task_credentials.list()\n\n# If the list is empty, create new task credentials\nclient.task_credentials.store()\n```\nThis will add new task credentials, which are required for deploying a custom foundation model on IBM watsonx.ai for IBM Cloud.",
   

<a id="runs"></a>
## Historical runs

In this section you learn to work with historical RAG Optimizer jobs (runs).

To list historical runs use the `list()` method and provide the `'rag_optimizer'` filter.

In [39]:
experiment.runs(filter='rag_optimizer').list()

,timestamp,run_id,state,auto_pipeline_optimizer name
0,2024-09-05T12:55:51.571Z,4322a167-b17b-4e24-8bb7-137c0770dd4b,completed,AutoAI RAG run - ModelInference documentation
1,2024-09-05T11:22:22.958Z,5f06cc01-9734-48e9-bbaf-657dfa16bef3,completed,AutoAI RAG run - ModelInference documentation
2,2024-09-05T08:46:06.437Z,9e9e289b-6302-49ff-a032-c0a98ccd89dc,completed,AutoAI RAG run - IBM Wikipedia
3,2024-08-22T09:13:05.513Z,d74d1d34-e81e-4ccd-804d-f17f33d3eccf,completed,RAG AutoAi name - sample notebook
4,2024-08-22T08:53:36.453Z,e52ff185-5702-4ea1-abee-ad6533196931,completed,RAG AutoAi name - sample notebook
5,2024-08-21T08:53:51.511Z,4f08fd2e-a7b1-4ef9-b368-d24b183f0c2f,completed,RAG AutoAi name - sample notebook
6,2024-08-12T09:49:17.588Z,55938b70-b5e5-4a8a-8ddd-13e2f44897fe,completed,RAG AutoAi name - sample notebook
7,2024-08-09T14:08:33.769Z,b2561f3a-6107-4029-a985-de1f78f8cd6b,completed,RAG AutoAi name - sample notebook
8,2024-08-09T12:06:45.972Z,61097ab3-723e-4aa8-bd30-7c007886ec50,completed,RAG AutoAi name - sample notebook


In [19]:
run_id = run_details['metadata']['id']
run_id

'b6315fa9-b65a-4fcb-a4e1-bf717d32c359'

### Get executed optimizer's configuration parameters

In [20]:
experiment.runs.get_rag_params(run_id=run_id)

{'name': 'AutoAI RAG run - ModelInference documentation',
 'description': 'AutoAI RAG Optimizer on ibm_watsonx_ai ModelInference documentation',
 'max_number_of_rag_patterns': 8,
 'optimization_metrics': ['answer_correctness']}

### Get historical rag_optimizer instance and training details

In [21]:
historical_opt = experiment.runs.get_rag_optimizer(run_id)

### List trained patterns for selected optimizer

In [22]:
historical_opt.summary()

,mean_answer_correctness,mean_faithfulness,mean_context_correctness,chunking.chunk_size,embeddings.model_id,vector_store.distance_metric,retrieval.method,retrieval.number_of_chunks,generation.model_id
Pattern_Name,,,,,,,,,
Pattern1,1.0,0.65940,1.0,512,ibm/slate-125m-english-rtrvr,euclidean,window,5,meta-llama/llama-3-70b-instruct
Pattern4,1.0,0.85365,1.0,1024,intfloat/multilingual-e5-large,cosine,window,5,meta-llama/llama-3-70b-instruct
Pattern6,1.0,0.73705,1.0,1024,intfloat/multilingual-e5-large,euclidean,window,3,meta-llama/llama-3-70b-instruct
Pattern8,1.0,0.87500,1.0,1024,ibm/slate-125m-english-rtrvr,euclidean,window,3,meta-llama/llama-3-70b-instruct
Pattern2,0.5,0.29950,1.0,1024,intfloat/multilingual-e5-large,euclidean,window,5,ibm/granite-13b-chat-v2
Pattern5,0.5,0.61920,1.0,1024,intfloat/multilingual-e5-large,cosine,window,3,mistralai/mixtral-8x7b-instruct-v01
Pattern7,0.5,0.29030,1.0,1024,ibm/slate-125m-english-rtrvr,cosine,window,5,ibm/granite-13b-chat-v2
Pattern3,0.0,0.71430,1.0,1024,intfloat/multilingual-e5-large,euclidean,simple,5,ibm/granite-13b-chat-v2


<a id="cleanup"></a>
## Clean up

To delete the current experiment, use the `cancel_run` method.

**Warning:** Be careful: once you delete an experiment, you will no longer be able to refer to it.

In [29]:
rag_optimizer.cancel_run()

'SUCCESS'

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!.

You learned how to use `ibm-watsonx-ai` to run AutoAI RAG experiments. 

 Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Mateusz Szewczyk**, Software Engineer watsonx.ai

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.